
## Table of Contents

- Introduction to Data Ingestion
- Text Files (.txt)
- PDF Documents
- Microsoft Word Documents
- CSV and Excel Files
- JSON and Structured Data
- Web Scraping
- Databases (SQL)
- Audio and Video Transcripts
- Advanced Techniques
- Best Practices

## Introduction to Data Ingestion

In [1]:
import os
from typing import List,Dict,Any
import pandas as pd

from langchain_core.documents import Document
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter,
    TokenTextSplitter   
)


## Understanding the document structure

- In Langchain the document structure is standardized Document object with page_content(a string), metadata(a dictionary) and an optional id(a string). This structure is used by components like document loaders and text splitters to represent, process and store external data in a consistent format, enabling applications to work with text and associated information

In [3]:
## Create a simple document

doc = Document(
    page_content="This is the mainn text content , I am venkata dharaneswara Reddy",
    metadata = {
        "source" : "example.txt",
        "page":1,
        "author":"Venkata Dharaneswara Reddy",
        "date_created":"2025-10-10",
        "custom_filed":"any_value"
    }
)

print("Document Structure")
print(f"Content : {doc.page_content}")
print(f"Metadata : {doc.metadata}")


Document Structure
Content : This is the mainn text content , I am venkata dharaneswara Reddy
Metadata : {'source': 'example.txt', 'page': 1, 'author': 'Venkata Dharaneswara Reddy', 'date_created': '2025-10-10', 'custom_filed': 'any_value'}


### Why Metadata Matters : 

1. Filtering search results
2. Tracking document sources
3. Providing context in responses
4. Debugging and auditing

In [4]:
print(type(doc))

<class 'langchain_core.documents.base.Document'>


### Reading a text File 

In [5]:
## create a single txt file
import os
os.makedirs("data/text_files",exist_ok=True)

In [7]:
sample_text = {
    "data/text_files/python_intro.txt":"""Python Programming Introduction

Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum and first released in 1991, Python has become one of the most popular
programming languages in the world.

Key Features:
- Easy to learn and use
- Extensive standard library
- Cross-platform compatibility
- Strong community support

Python is widely used in web development, data science, artificial intelligence, and automation.""",
    
    "data/text_files/machine_learning.txt": """Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables systems to learn and improve
from experience without being explicitly programmed. It focuses on developing computer programs
that can access data and use it to learn for themselves.

Types of Machine Learning:
1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
3. Reinforcement Learning: Learning through rewards and penalties

Applications include image recognition, speech processing, and recommendation systems
    """
}


for file_path,content in sample_text.items():
    with open(file_path,'w',encoding="utf-8") as file:
        file.write(content)
        
print("Sample Text File Created")

Sample Text File Created


### Text Loader - Read Single File

In [3]:
from langchain.document_loaders import TextLoader


# Loading a single text file
loader = TextLoader("data/text_files/python_intro.txt",encoding="utf-8")
documents = loader.load()
print(type(documents))
print(documents)
print()
print(f"Content Preview: {documents[0].page_content[:100]}...")
print(f"Meda data: {documents[0].metadata}")

<class 'list'>
[Document(metadata={'source': 'data/text_files/python_intro.txt'}, page_content='Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular\nprogramming languages in the world.\n\nKey Features:\n- Easy to learn and use\n- Extensive standard library\n- Cross-platform compatibility\n- Strong community support\n\nPython is widely used in web development, data science, artificial intelligence, and automation.')]

Content Preview: Python Programming Introduction

Python is a high-level, interpreted programming language known for ...
Meda data: {'source': 'data/text_files/python_intro.txt'}


### Directory loader - suitable for reading multiple text files 

In [4]:
from langchain_community.document_loaders import DirectoryLoader

### load all the text files from the directory

dir_loader = DirectoryLoader(
    "data/text_files",
    glob="**/*.txt", # pattern matching  
    loader_cls=TextLoader,
    loader_kwargs={"encoding":"utf-8"},
    show_progress=True
)

documents = dir_loader.load()


print(f"Loaded {len(documents)} documents")
for i, doc in enumerate(documents):
    print(f"\nDocuments {i + 1}:")
    print(f"Source: {doc.metadata['source']}")
    print(f" Length: {len(doc.page_content)} characters")


100%|██████████| 2/2 [00:00<00:00, 2516.83it/s]

Loaded 2 documents

Documents 1:
Source: data/text_files/python_intro.txt
 Length: 489 characters

Documents 2:
Source: data/text_files/machine_learning.txt
 Length: 573 characters


## DirectoryLoader Characteristics: 

### Advantages: 

1. Loads multiple files at once.
2. supports glob patterns.
3. Progress Tracking.
4. Recursive Directory Scanning.

### Disadvantages: 

1. All files must be same type. 
2. Limited Error Handlinng Per File.
3. Can be memory intensive for large directories.

### Text Splitting Strategies

In [5]:
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
    TokenTextSplitter
)

print(documents)

[Document(metadata={'source': 'data/text_files/python_intro.txt'}, page_content='Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular\nprogramming languages in the world.\n\nKey Features:\n- Easy to learn and use\n- Extensive standard library\n- Cross-platform compatibility\n- Strong community support\n\nPython is widely used in web development, data science, artificial intelligence, and automation.'), Document(metadata={'source': 'data/text_files/machine_learning.txt'}, page_content='Machine Learning Basics\n\nMachine learning is a subset of artificial intelligence that enables systems to learn and improve\nfrom experience without being explicitly programmed. It focuses on developing computer programs\nthat can access data and use it to learn for themselves.\n\nTypes of Machine Learning:\n1. Supervised Learn

### Method 1 - Character Text Splitter 



In [9]:
text = documents[1].page_content


print("Character text splitter")
char_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size = 200,
    chunk_overlap = 20,
    length_function = len
)


char_chunks = char_splitter.split_text(text)
print(f"Created {len(char_chunks)} chunks")
print(f"First chunk: {char_chunks[0][:100]}...")

Character text splitter
Created 4 chunks
First chunk: Machine Learning Basics
Machine learning is a subset of artificial intelligence that enables systems...


In [11]:
print(char_chunks[0])
print()
print(char_chunks[1])
print()
print(char_chunks[2])
print()
print(char_chunks[3])

Machine Learning Basics
Machine learning is a subset of artificial intelligence that enables systems to learn and improve

from experience without being explicitly programmed. It focuses on developing computer programs
that can access data and use it to learn for themselves.
Types of Machine Learning:

1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
3. Reinforcement Learning: Learning through rewards and penalties

Applications include image recognition, speech processing, and recommendation systems


### Method 2 - Recursive Character Splitter (Most Recommended Method)

In [12]:
recursive_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n","\n"," ",""],
    chunk_size = 200,
    chunk_overlap = 20,
    length_function = len
)

recursive_chunks = recursive_splitter.split_text(text)
print(f"Created {len(recursive_chunks)} chunks")
print(f"First Chunk: {recursive_chunks[0][:100]}...")

Created 6 chunks
First Chunk: Machine Learning Basics...


In [13]:
print(recursive_chunks[0])
print("-------------------")
print(recursive_chunks[1])
print("-------------------")
print(recursive_chunks[2])
print("-------------------")
print(recursive_chunks[3])
print("-------------------")
print(recursive_chunks[4])
print("-------------------")
print(recursive_chunks[5])

Machine Learning Basics
-------------------
Machine learning is a subset of artificial intelligence that enables systems to learn and improve
from experience without being explicitly programmed. It focuses on developing computer programs
-------------------
that can access data and use it to learn for themselves.
-------------------
Types of Machine Learning:
1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
-------------------
3. Reinforcement Learning: Learning through rewards and penalties
-------------------
Applications include image recognition, speech processing, and recommendation systems


In [17]:
simple_text = "This is sentence one and it is quite long. This is sentence two and it is also quite long. This is sentence three which is even longer than the others. This is sentence four. This is sentence five. This is sentence six."

splitter = RecursiveCharacterTextSplitter(
    separators=[" "],
    chunk_size = 80,
    chunk_overlap = 20,
    length_function = len
)

chunks = splitter.split_text(simple_text)

print(f"\nSimple text example - {len(chunks)} chunks:\n")

for i in range(len(chunks) - 1):
    print(f"Chunk {i+1}: '{chunks[i]}'")
    print(f"Chunk {i+2}: '{chunks[i+1]}'")
    
    
    print()
    
    
# we can see overlap here because of the seperators (only using " ")
# before we are using many seperators 


Simple text example - 4 chunks:

Chunk 1: 'This is sentence one and it is quite long. This is sentence two and it is also'
Chunk 2: 'two and it is also quite long. This is sentence three which is even longer than'

Chunk 2: 'two and it is also quite long. This is sentence three which is even longer than'
Chunk 3: 'is even longer than the others. This is sentence four. This is sentence five.'

Chunk 3: 'is even longer than the others. This is sentence four. This is sentence five.'
Chunk 4: 'is sentence five. This is sentence six.'



### Method 3 : Token Based Splitting

In [19]:
token_splitter = TokenTextSplitter(
    chunk_size = 50,
    chunk_overlap = 10
)

token_chunks = token_splitter.split_text(text)
print(f"Created: {len(token_chunks)} chunks")
print(f"First Chunk: {token_chunks[0][:100]}")

Created: 3 chunks
First Chunk: Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables system


### Comparison

#### CharacterTextSplitter:
■ Simple and predictable  
■ Good for structured text  
■ May break mid-sentence  
*Use when*: Text has clear delimiters

#### RecursiveCharacterTextSplitter:
■ Respects text structure  
■ Tries multiple separators  
■ Best general-purpose splitter  
■ Slightly more complex  
*Use when*: Default choice for most texts

#### TokenTextSplitter:
■ Respects model token limits  
■ More accurate for embeddings  
■ Slower than character-based  
*Use when*: Working with token-limited models
